In [1]:
import nltk
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import pylab as pl
nltk.download()
def preprocess(tweet):
    
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    
    #Convert @username to __USERHANDLE
    tweet = re.sub('@[^\s]+','__USERHANDLE',tweet)  
    
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    
    #trim
    tweet = tweet.strip('\'"')
    
    # Repeating words like hellloooo
    repeat_char = re.compile(r"(.)\1{1,}", re.IGNORECASE)
    tweet = repeat_char.sub(r"\1\1", tweet)
    
    #Emoticons
    emoticons = \
    [
     ('__positive__',[ ':-)', ':)', '(:', '(-:', \
                       ':-D', ':D', 'X-D', 'XD', 'xD', \
                       '<3', ':\*', ';-)', ';)', ';-D', ';D', '(;', '(-;', ] ),\
     ('__negative__', [':-(', ':(', '(:', '(-:', ':,(',\
                       ':\'(', ':"(', ':((','D:' ] ),\
    ]

    def replace_parenthesis(arr):
       return [text.replace(')', '[)}\]]').replace('(', '[({\[]') for text in arr]
    
    def join_parenthesis(arr):
        return '(' + '|'.join( arr ) + ')'

    emoticons_regex = [ (repl, re.compile(join_parenthesis(replace_parenthesis(regx))) ) \
            for (repl, regx) in emoticons ]
    
    for (repl, regx) in emoticons_regex :
        tweet = re.sub(regx, ' '+repl+' ', tweet)

     #Convert to lower case
    tweet = tweet.lower()
    
    return tweet

#Stemming of Tweets

def stem(tweet):
        stemmer = nltk.stem.PorterStemmer()
        tweet_stem = ''
        words = [word if(word[0:2]=='__') else word.lower() \
                    for word in tweet.split() \
                    if len(word) >= 3]
        words = [stemmer.stem(w) for w in words] 
        tweet_stem = ' '.join(words)
        return tweet_stem

dataset = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', skiprows=785000, nrows=30000, encoding='ISO-8859-1',header=None)
X=dataset.iloc[:,5].values
X=pd.Series(X)
y=dataset.iloc[:,0].values
'''
for row in range(0,1600000):
    if y[row]==4:
        y[row]=1
    else:
        y[row]=0
'''
        


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... Brown Corpus
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat............ CESS-CAT Treebank
  [

'\nfor row in range(0,1600000):\n    if y[row]==4:\n        y[row]=1\n    else:\n        y[row]=0\n'

In [0]:

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=5)

X_train = [stem(preprocess(tweet)) for tweet in X_train]
X_test = [stem(preprocess(tweet)) for tweet in X_test]

vec = TfidfVectorizer(min_df=5, max_df=0.95, sublinear_tf = True,use_idf = True,ngram_range=(1, 2))
X_train_vec = vec.fit_transform(X_train)
X_test_vec = vec.transform(X_test)
#nb = MultinomialNB()








In [3]:
from sklearn.svm import SVC
svclassifier = SVC(kernel = 'linear')

svclassifier.fit(X_train_vec, y_train)



SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [4]:
log_predicted= svclassifier.predict(X_test_vec)

logreg_score = round(svclassifier.score(X_train_vec, y_train) * 100, 2)
logreg_score_test = round(svclassifier.score(X_test_vec, y_test) * 100, 2)

print('Logistic Regression Training Score: \n', logreg_score)
print('Logistic Regression Test Score: \n', logreg_score_test)
print('Coefficient: \n', svclassifier.coef_)
print('Intercept: \n', svclassifier.intercept_)
print('Accuracy: \n', metrics.accuracy_score(y_test,log_predicted))
print('Confusion Matrix: \n', metrics.confusion_matrix(y_test,log_predicted))
print('Classification Report: \n', metrics.classification_report(y_test,log_predicted))


# nb.fit(X_train_vec,y_train)
# X_test_vec = vec.transform(X_test)
# pred = nb.predict(X_test_vec)

# print(metrics.accuracy_score(y_test, pred))
# y_test.mean()
# # Out[9]: 0.500659375
# 1-y_test.mean()
# # Out[10]: 0.499340625

# print(metrics.confusion_matrix(y_test, pred))
# #[[128420  31369]
# # [ 33164 127047]]
# cm = metrics.confusion_matrix(y_test, pred)
# pl.matshow(cm)
# #pl.title('Confusion matrix of the classifier')
# pl.colorbar()
# pl.show()

# print(metrics.classification_report(y_test, pred))

Logistic Regression Training Score: 
 86.6
Logistic Regression Test Score: 
 78.25
Coefficient: 
   (0, 7353)	0.27265536646815336
  (0, 9006)	0.23610042024668235
  (0, 2496)	0.37922873950154534
  (0, 7919)	0.2883785862503066
  (0, 8606)	0.1336221000122825
  (0, 3189)	0.32654948360533764
  (0, 505)	0.2972498510584681
  (0, 5473)	0.3558164125699872
  (0, 6838)	0.3201336242653064
  (0, 2371)	0.35486347091049747
  (0, 7029)	0.003430894276115387
  (0, 8573)	0.33410187654263157
  (0, 6417)	0.624309912571683
  (0, 280)	0.315777843086848
  (0, 5888)	0.5035327490439919
  (0, 3482)	0.2688785323254924
  (0, 3064)	0.3080025076521876
  (0, 496)	0.5630296532716151
  (0, 2290)	0.4079187252027927
  (0, 3206)	0.33175002344261983
  (0, 8091)	0.5570280905875891
  (0, 4978)	0.35864963013717377
  (0, 3010)	0.3549398590644984
  (0, 1024)	0.6608720224766205
  (0, 4976)	0.30729283815812036
  :	:
  (0, 1248)	-0.2534845003567743
  (0, 1146)	0.4391844495405341
  (0, 416)	0.05626553954335789
  (0, 242)	0.31690309